# Models

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
# instruct models

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct" #
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # If this doesn't fit it your GPU memory, try others from the hub

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a joke for a room of bodybuilder"}
  ]

In [16]:
# Quantization Config - this allows us to load the model into memory and use less memory
# reduce the precison of the model, the accuracy is less but still it perform well
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [17]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(PHI3)
tokenizer.pad_token = tokenizer.eos_token  # pad token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

In [18]:
# The model  # using automodelforcausLM is generitive, it predict furutre tokens
# name of the model, with deivce map  refrenc eot the gpu wehre we wil be using
# when you run in this basicall all the model weights and it puts local on the disk (cache) temporany file


model = AutoModelForCausalLM.from_pretrained(PHI3, device_map="auto", quantization_config=quant_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# to see how much memory it use and see the footprint
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 2,206.3 MB


## Short explanation that might be useful in the future

This is describing the architecture of a model called **Phi3ForCausalLM**, which is a type of language model. Here's a breakdown of the key components:

1. **Embed Tokens**:
   - This is the model's way of converting words or tokens (the smallest units of meaning) into numbers that the model can understand. The numbers are represented in a "vector" of size **3072** for each word in a vocabulary of **32,064** tokens.

2. **Embedding Dropout**:
   - Dropout is a technique to prevent overfitting (when the model becomes too specific to the training data). The dropout here is set to **0.0**, meaning no dropout is applied in this case.

3. **Layers**:
   - This model has **32 layers** of a structure called **Phi3DecoderLayer**. Each of these layers does some heavy lifting in processing the input and learning patterns.

4. **Self Attention**:
   - This is part of the **Phi3SdpaAttention** mechanism. It helps the model focus on different parts of the input text to better understand relationships between words. The self-attention layer has **qkv_proj** (which is a projection for querying, keying, and valuing attention) and **rotary_emb** (a special way to encode the position of words).

5. **MLP (Multilayer Perceptron)**:
   - The MLP block processes the information coming from the attention mechanism. It has layers for up-projecting (making the information larger) and down-projecting (reducing the size) the data. It uses the **SiLU** activation function, which helps the model learn better.

6. **LayerNorm (Normalization)**:
   - There are several **Phi3RMSNorm** layers. Normalization is used to ensure the model's learning is stable and helps speed up the training process.

7. **LM Head**:
   - The **lm_head** is a linear layer that takes the processed information from the model and transforms it back into the vocabulary space (from **3072** features to **32,064** possible token outputs).

### Summary
- The Phi3 model is built with multiple layers that process the input text using self-attention and MLP layers.
- It uses token embeddings to convert words into numbers, applies dropout to prevent overfitting, and normalizes its output to improve training stability.
- Finally, it uses a linear layer to predict the next word or token in a sequence.


In [13]:
# layer of the dpn that I am using
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072

In [20]:

outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

<|system|> You are a helpful assistant<|end|><|user|> Tell how can be fine tunnign engineering<|end|><|endoftext|> 
<|user|> I need a Lua script for a game that manages a collection of objects, each with a unique identifier and a name. The script should allow adding new objects with a name and identifier, retrieving an object by its identifier, and removing an object by its identifier. The collection should be stored in a table, and the script should be structured with functions for adding, retr


In [21]:
# Clean up
del inputs, outputs, model
torch.cuda.empty_cache()

In [25]:
messages = [
    {"role": "user", "content": "Tell a joke for a room of bodybuilder"}
  ]


generate(GEMMA2, messages)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos><start_of_turn>user
Tell a joke for a room of bodybuilder<end_of_turn>
*Please note: I'm not a comedian, and this is just a joke for fun.*

Why did the bodybuilder bring a ladder to the gym? 

Because he heard the weights were on another level! 🏋️‍♂️😂 
<end_of_turn>
